## Import and Configuration

https://platform.openai.com/

In [ ]:
import os
import re
import csv
import copy
import json
import math
import openai
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from datetime import datetime
from collections import defaultdict, Counter
from matplotlib.colors import ListedColormap

In [ ]:
### Configurations

TIME_LIMIT = 60 # seconds

DEFAULT_SYS_MSG = "You are a helpful assistant."
SYS_MSG = DEFAULT_SYS_MSG

MODEL = 'gpt4'
USE_CHAT = True

def switch_model(model):
    if model == 'turbo': model = 'gpt-3.5-turbo-0301'
    if model == 'turbo-0613': model = 'gpt-3.5-turbo-0613'
    if model == 'turbo-1106': model = 'gpt-3.5-turbo-1106'
    if model == 'gpt3': model = 'text-davinci-003'
    if model == 'gpt4': model = 'gpt-4-0314'
    if model == 'gpt4-0613': model = 'gpt-4-0613'
    if model == 'gpt4-1106': model = 'gpt-4-1106-preview'

    global MODEL, USE_CHAT
    MODEL = model
    if 'gpt-4' in model:
        openai.api_key = 'YOUR_API_KEY'
    else: 
        openai.api_key = 'YOUR_API_KEY'

    if 'text' in model:
        USE_CHAT = False
    else:
        USE_CHAT = True

switch_model('gpt4')

In [ ]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


## Functions

### Run

In [ ]:
def get_response(
    messages: list = [],
    model=None,
    use_chat=None,
    **kwargs
) -> dict:
    if model is None: model = MODEL
    if use_chat is None: use_chat = USE_CHAT
    # print('model:', model, MODEL)
    with time_limit(TIME_LIMIT):
        if use_chat:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                **kwargs
            ).to_dict_recursive()
        else:
            prompt = '\n\n'.join([x['content'] for x in messages]) + '\n\n'
            response = openai.Completion.create(
                model=model,
                prompt=prompt,
                max_tokens=100,
                **kwargs
            ).to_dict_recursive()

    return response

def update_messages(
    messages, 
    responses, 
    prompt,
    **kwargs
):
    messages.append(
        {"role": "user", "content": prompt}
    )
    response = get_response(messages, **kwargs)
    responses.append(response)
    messages.append(
        response['choices'][0]['message'] if USE_CHAT else {
            'role': 'assistant',
            'content': response['choices'][0]['text']
        }
    )

In [ ]:
def run_one_session(
    prompts=[], 
    n_instances=30, 
    print_except=True,
    orders=None,
    tqdm_silent=False,
    **kwargs
):
    # global records
    records = {
        'messages' : [],
        'responses' : [],
    }
    if orders is None:
        orders = [list(range(len(prompts)))] * n_instances
    with tqdm(total=n_instances, disable=tqdm_silent) as pbar:
        i = 0
        while i < n_instances:
        # while pbar.n < n_instances:
            try:
                responses = []
                messages = [
                    {"role": "system", "content": SYS_MSG},
                ]
                for prompt_id in orders[i]:
                # for prompt_id in orders[pbar.n]:
                    prompt = prompts[prompt_id]
                    update_messages(
                        messages, 
                        responses, 
                        prompt,
                        **kwargs
                    )
                records['messages'].append(messages)
                records['responses'].append(responses)

                i += 1
                # pbar.update(1)
                
            except Exception as e: 
                if print_except: print(e)
                continue
    return records

### Extract 

In [ ]:
def extract_brackets(text, brackets='[]'):
    assert len(brackets) == 2
    pattern = re.escape(brackets[0]) + r'(.*?)' + re.escape(brackets[1])
    matches = re.findall(pattern, text)
    return matches

def extract_amout(
    message, 
    prefix='',
    print_except=True,
    type=float,
    brackets='[]'
):
    try:
        matches = extract_brackets(message, brackets=brackets)
        matches = [s.replace(' ', '') for s in matches]
        matches = [s[len(prefix):] \
            if s.startswith(prefix) \
            else s for s in matches]
        invalid = False
        if len(matches) == 0:
            invalid = True
        for i in range(len(matches)):
            if matches[i] != matches[0]:
                invalid = True
        if invalid:
            raise ValueError('Invalid answer: %s' % message)
        return type(matches[0])
    except Exception as e: 
        if print_except: print(e)
        return None

def extract_choices(records):
    choices = [extract_amout(
        messages[-1]['content'], 
        prefix='$', 
        print_except=True,
        type=float) for messages in records['messages']
    ]
    choices = [x for x in choices if x is not None]
    # print(choices)
    return choices

In [ ]:
def extract_chatgpt(
    message, 
    extractor,
    print_except=True,
    type=float,
):
    prompts = [extractor(message)]
    records = run_one_session(
        prompts, 
        n_instances=1, 
        print_except=print_except,
        tqdm_silent=True,
        use_chat=True,
        model='gpt-3.5-turbo-0301',
    )
    extracted = records['messages'][0][-1]['content']
    extracted = extracted.replace(' ', '')
    try:
        if 'None' in extracted:
            raise ValueError('Invalid answer: %s' % message)
        extracted = type(extracted)
    except Exception as e:
        if print_except: 
            raise e
        extracted = None
    return extracted

### Test and Save

In [ ]:
def unit_test(
    records=None, 
    n_instances=1, 
    run_func=lambda x: run_one_session(prompts, n_instances=x),
):
    if records is None:
        records = run_func(n_instances)
    for i, messages in enumerate(records['messages']):
        print('===== Instance #%i' % i)
        for item in messages:
            if item['role'] != 'assistant': continue
            print('> ', item['content'])

In [ ]:
# prompts = [
#     'Hi, let’s play a game. ',
#     'You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). ',
# ]

# unit_test(n_instances=1)

In [ ]:
def save_records(game_name, records):
    timestamp = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
    file_name = '%s_%s.json' % (game_name, timestamp)

    with open(os.path.join('records', file_name), 'w') as f:
        json.dump(records, f)

### Plot

In [ ]:
def choices_to_df(choices, hue):
    df = pd.DataFrame(choices, columns=['choices'])
    df['hue'] = hue
    df['hue'] = df['hue'].astype(str)
    return df

def plot(
        df, 
        title='',
        x='choices',
        hue='hue',
        binrange=None, 
        binwidth=None,
        stat='count',
        multiple='dodge'
    ):
    if binrange is None:
        binrange = (df[x].min(), df[x].max())
    df = df.dropna(axis=0, subset=[x]).reset_index()
    plt.figure(figsize=(8, 6))
    ax = sns.histplot(
        data=df, 
        x=x,
        hue=hue, 
        kde=True,
        binrange=binrange, 
        binwidth=binwidth,
        stat=stat,
        multiple=multiple,
        shrink=.8,
    )
    ax.set_title(title)
    return ax

def plot_facet(
    df_list,
    x='choices',
    hue='hue',
    binrange=None,
    binwidth=10,
    stat='count',
    x_label='',
    sharex=True,
    sharey=False,
):
    data = pd.concat(df_list)
    if binrange is None:
        binrange = (data[x].min(), data[x].max())
    g = sns.FacetGrid(
        data, row=hue, hue=hue, 
        aspect=3, height=2, 
        sharex=sharex, sharey=sharey,
        despine=False
    )
    g.map_dataframe(
        sns.histplot, 
        x=x, 
        kde=True, 
        binrange=binrange, 
        binwidth=binwidth,
        stat=stat,
    )
    # g.add_legend(title='hue')
    g.set_axis_labels(x_label, stat)
    return g

### Occupations

In [ ]:
# OCCUPATIONS = ['mathematician', 'public relations specialist', 'journalist', 'investment fund manager', 'game theorist', 'teacher', 'politician']
OCCUPATIONS = {
    'mathematician': {
        'core' : [
            'Address the relationships of quantities, magnitudes, and forms through the use of numbers and symbols.',
            'Disseminate research by writing reports, publishing papers, or presenting at professional conferences.',
            'Maintain knowledge in the field by reading professional journals, talking with other mathematicians, and attending professional conferences.',
            'Apply mathematical theories and techniques to the solution of practical problems in business, engineering, the sciences, or other fields.',
            'Conduct research to extend mathematical knowledge in traditional areas, such as algebra, geometry, probability, and logic.',
            'Develop mathematical or statistical models of phenomena to be used for analysis or for computational simulation.',
            'Perform computations and apply methods of numerical analysis to data.',
            'Assemble sets of assumptions, and explore the consequences of each set.',
            'Develop new principles and new relationships between existing mathematical principles to advance mathematical science.',
            'Develop computational methods for solving problems that occur in areas of science and engineering or that come from applications in business or industry.',
        ],
        'supplemental': [
            'Design, analyze, and decipher encryption systems designed to transmit military, political, financial, or law-enforcement-related information in code.',
        ]
    },
    'public relations specialist': {
        'core': [
            'Respond to requests for information from the media or designate an appropriate spokesperson or information source.',
            'Plan or direct development or communication of programs to maintain favorable public or stockholder perceptions of an organization\'s accomplishments, agenda, or environmental responsibility.',
            'Post and update content on the company\'s Web site and social media outlets.',
            'Write press releases or other media communications to promote clients.',
            'Establish or maintain cooperative relationships with representatives of community, consumer, employee, or public interest groups.',
            'Confer with other managers to identify trends or key group interests or concerns or to provide advice on business decisions.',
            'Coach client representatives in effective communication with the public or with employees.',
            'Study the objectives, promotional policies, or needs of organizations to develop public relations strategies that will influence public opinion or promote ideas, products, or services.',
            'Prepare or edit organizational publications, such as employee newsletters or stockholders\' reports, for internal or external audiences.',
            'Arrange public appearances, lectures, contests, or exhibits for clients to increase product or service awareness or to promote goodwill.',
            'Plan or conduct market or public opinion research to test products or determine potential for product success, communicating results to client or management.',
            'Develop plans or materials to communicate organizational activities that are beneficial to the environment, public safety, or other important social issues.',
            'Confer with production or support personnel to produce or coordinate production of advertisements or promotions.',
            'Consult with advertising agencies or staff to arrange promotional campaigns in all types of media for products, organizations, or individuals.',
            'Prepare or deliver speeches to further public relations objectives.',
        ],
        'supplemental': [
            'Coordinate public responses to environmental management incidents or conflicts.',
            'Develop marketing campaigns for environmental technologies or services.',
            'Purchase advertising space or time as required to promote client\'s product or agenda.',
        ]
    },
    'journalist': {
        'core': [
            'Write commentaries, columns, or scripts, using computers.',
            'Coordinate and serve as an anchor on news broadcast programs.',
            'Examine news items of local, national, and international significance to determine topics to address, or obtain assignments from editorial staff members.',
            'Analyze and interpret news and information received from various sources to broadcast the information.',
            'Receive assignments or evaluate leads or tips to develop story ideas.',
            'Research a story\'s background information to provide complete and accurate information.',
            'Arrange interviews with people who can provide information about a story.',
            'Gather information and develop perspectives about news subjects through research, interviews, observation, and experience.',
            'Select material most pertinent to presentation, and organize this material into appropriate formats.',
            'Present news stories, and introduce in-depth videotaped segments or live transmissions from on-the-scene reporters.',
            'Establish and maintain relationships with individuals who are credible sources of information.',
            'Report news stories for publication or broadcast, describing the background and details of events.',
            'Revise work to meet editorial approval or to fit time or space requirements.',
            'Review and evaluate notes taken about news events to isolate pertinent facts and details.',
            'Investigate breaking news developments, such as disasters, crimes, or human-interest stories.',
            'Review written, audio, or video copy, and correct errors in content, grammar, or punctuation, following prescribed editorial style and formatting guidelines.',
            'Report on specialized fields such as medicine, green technology, environmental issues, science, politics, sports, arts, consumer affairs, business, religion, crime, or education.',
            'Determine a published or broadcasted story\'s emphasis, length, and format, organizing material accordingly.',
            'Transmit news stories or reporting information from remote locations, using equipment such as satellite phones, telephones, fax machines, or modems.',
            'Check reference materials, such as books, news files, or public records, to obtain relevant facts.',
            'Discuss issues with editors to establish priorities or positions.',
            'Photograph or videotape news events.',
            'Present live or recorded commentary via broadcast media.',
            'Take pictures or video, and process them for inclusion in a story.',
            'Conduct taped or filmed interviews or narratives.',
            'Develop ideas or material for columns or commentaries by analyzing and interpreting news, current issues, or personal experiences.',
            'Communicate with readers, viewers, advertisers, or the general public via mail, email, or telephone.',
            'Write online blog entries that address news developments or offer additional information, opinions, or commentary on news events.',
            'Assign stories to other reporters or duties to production staff.',
            'Write columns, editorials, commentaries, or reviews that interpret events or offer opinions.',
        ],
    },
    'investment fund manager': {
        'core': [
            'Manage investment funds to maximize return on client investments.',
            'Select specific investments or investment mixes for purchase by an investment fund.',
            'Monitor financial or operational performance of individual investments to ensure portfolios meet risk goals.',
            'Select or direct the execution of trades.',
            'Develop or implement fund investment policies or strategies.',
            'Perform or evaluate research, such as detailed company or industry analyses, to inform financial forecasting, decision making, or valuation.',
            'Present investment information, such as product risks, fees, or fund performance statistics.',
            'Develop, implement, or monitor security valuation policies.',
            'Meet with investors to determine investment goals or to discuss investment strategies.',
            'Attend investment briefings or consult financial media to stay abreast of relevant investment markets.',
            'Prepare for and respond to regulatory inquiries.',
            'Evaluate the potential of new product developments or market opportunities, according to factors such as business plans, technologies, or market potential.',
            'Hire or evaluate staff.',
            'Monitor regulatory or tax law changes to ensure fund compliance or to capitalize on development opportunities.',
            'Develop or direct development of offering documents or marketing materials.',
        ],
        'supplemental': [
            'Analyze acquisitions to ensure conformance with strategic goals or regulatory requirements.',
            'Verify regulatory compliance of transaction reporting.',
            'Review offering documents or marketing materials to ensure regulatory compliance.',
            'Identify group or individual target investors for a specific fund.',
            'Direct activities of accounting or operations departments.',
        ]
    },
    'economist': {
        'core': [
            'Study economic and statistical data in area of specialization, such as finance, labor, or agriculture.',
            'Conduct research on economic issues, and disseminate research findings through technical reports or scientific articles in journals.',
            'Compile, analyze, and report data to explain economic phenomena and forecast market trends, applying mathematical models and statistical techniques.',
            'Supervise research projects and students\' study projects.',
            'Teach theories, principles, and methods of economics.',
            'Study the socioeconomic impacts of new public policies, such as proposed legislation, taxes, services, and regulations.',
            'Formulate recommendations, policies, or plans to solve economic problems or to interpret markets.',
            'Explain economic impact of policies to the public.',
            'Provide advice and consultation on economic relationships to businesses, public and private agencies, and other employers.',
            'Forecast production and consumption of renewable resources and supply, consumption, and depletion of non-renewable resources.',
            'Develop economic guidelines and standards, and prepare points of view used in forecasting trends and formulating economic policy.',
        ],
        'supplemental': [
            'Testify at regulatory or legislative hearings concerning the estimated effects of changes in legislation or public policy, and present recommendations based on cost-benefit analyses.',
            'Provide litigation support, such as writing reports for expert testimony or testifying as an expert witness.',
        ]
    },
    'legislator': {
        'core': [
            'Analyze and understand the local and national implications of proposed legislation.',
            'Appoint nominees to leadership posts, or approve such appointments.',
            'Confer with colleagues to formulate positions and strategies pertaining to pending issues.',
            'Debate the merits of proposals and bill amendments during floor sessions, following the appropriate rules of procedure.',
            'Develop expertise in subject matters related to committee assignments.',
            'Hear testimony from constituents, representatives of interest groups, board and commission members, and others with an interest in bills or issues under consideration.',
            'Keep abreast of the issues affecting constituents by making personal visits and phone calls, reading local newspapers, and viewing or listening to local broadcasts.',
            'Maintain knowledge of relevant national and international current events.',
            'Make decisions that balance the perspectives of private citizens, public officials, and party leaders.',
            'Negotiate with colleagues or members of other political parties in order to reconcile differing interests, and to create policies and agreements.',
            'Prepare drafts of amendments, government policies, laws, rules, regulations, budgets, programs and procedures.',
            'Read and review concerns of constituents or the general public and determine if governmental action is necessary.',
            'Represent their parties in negotiations with political executives or members of other parties, and when speaking with the media.',
            'Review bills in committee, and make recommendations about their future.',
            'Seek federal funding for local projects and programs.',
            'Serve on commissions, investigative panels, study groups, and committees in order to examine specialized areas and recommend action.',
            'Vote on motions, amendments, and decisions on whether or not to report a bill out from committee to the assembly floor.',
            'Write, prepare, and deliver statements for the Congressional Record.',
            'Alert constituents of government actions and programs by way of newsletters, personal appearances at town meetings, phone calls, and individual meetings.',
            'Attend receptions, dinners, and conferences to meet people, exchange views and information, and develop working relationships.',
            'Conduct "head counts" to help predict the outcome of upcoming votes.',
            'Determine campaign strategies for media advertising, positions on issues, and public appearances.',
            'Encourage and support party candidates for political office.',
            'Establish personal offices in local districts or states, and manage office staff.',
            'Evaluate the structure, efficiency, activities, and performance of government agencies.',
            'Organize and maintain campaign organizations and fundraisers, in order to raise money for election or re-election.',
            'Oversee expense allowances, ensuring that accounts are balanced at the end of each fiscal year.',
            'Promote the industries and products of their electoral districts.',
            'Represent their government at local, national, and international meetings and conferences.',
            'Speak to students to encourage and support the development of future political leaders.',
        ],
    },
}

In [ ]:
occupation = 'mathematician'
prompt = '''You are a {}.\n
Your core tasks include:\n* {}'
'''.format(
    occupation, 
    '\n* '.join(OCCUPATIONS[occupation]['core']), 
) + ('''\n\nYour supplemental tasks include:\n* {}
'''.format(
    '\n* '.join(OCCUPATIONS[occupation]['supplemental'])
) if 'supplemental' in OCCUPATIONS[occupation] else '')
print(prompt)

In [ ]:
def run_occupations(
    occupations=OCCUPATIONS,
    run_func=run_one_session,
    extract_func=extract_choices,
    **kwargs,
):
    global SYS_MSG
    global records_all
    global choices_all
    records_all = {}
    choices_all = {}
    for occupation in occupations:
        print('occupation: ', occupation)
        
        # SYS_MSG = 'You are a %s. ' % occupation
        SYS_MSG = '''You are a {}.\n
        Your core tasks include:\n* {}'
        '''.format(
            occupation, 
            '\n* '.join(OCCUPATIONS[occupation]['core']), 
        ) + ('''\n\nYour supplemental tasks include:\n* {}
        '''.format(
            '\n* '.join(OCCUPATIONS[occupation]['supplemental'])
        ) if 'supplemental' in OCCUPATIONS[occupation] else '')

        records = run_func(**kwargs)

        choices = extract_func(records)
        choices_all[occupation] = choices
        records_all[occupation] = records
    return records_all, choices_all

In [ ]:
def plot_occupations(
    df_baseline,
    choices_all,
    binrange=(0,100),
    binwidth=5,
    x_label='$ to give',
    stat='density',
):
    print('baseline: ', len(df_baseline))
    df_list = []
    for occupation in choices_all:
        choices = choices_all[occupation]
        print(occupation, ':', len(choices))
        df = choices_to_df(choices, hue=occupation)
        df_list.append(df)
    plot_facet(
        df_list=[df_baseline]+df_list,
        binrange=binrange,
        binwidth=binwidth,
        x_label=x_label,
        stat=stat,
    )

## Big Five Test

In [ ]:
questions = {}
with open('data/bigfive.tsv', 'r') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        questions[row[0]] = row[1]

keyed = {}
indices = defaultdict(int)
dimensions = 'EACNO'
with open('data/bigfive_IPIP.tsv', 'r') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        d = dimensions[int(row[-1][1])-1]
        v = row[-1][2]
        indices[d] += 1
        k = '%s%i' % (d, indices[d])
        keyed[k] = v

In [ ]:
def run_bigfive(
    n_instances=30, 
    print_except=False,
):
    global records
    records = {
        'messages' : [],
        'responses' : [],
        'choices' : [],
    }
    cnt_invalid = {}
    for _ in range(n_instances):
        messages = [
            {"role": "system", "content": ''},
        ]
        responses = []
        choices = []
        for i, q in tqdm(questions.items()):
            prompt = 'The following item was rated on a five point scale where 1=Disagree, 2=Partially Disagree, 3=Neutral, 4=Partially Agree, 5=Agree. Please select how the statement describes you and highlight your answer in [] (such as [1], [2], [3], [4], or [5]). \n' + q
            
            while True:
                try:
                    messages_copy = messages.copy()
                    responses_copy = responses.copy()
                    update_messages(messages, responses, prompt)
                    choice = extract_amout(
                        messages[-1]['content'], 
                        print_except=print_except,
                        type=int
                    )
                    if choice is None or \
                        choice < 1 or choice > 5:
                        raise ValueError(
                            'Invalid answer: %s' % messages[-1]['content'])
                    choices.append(choice)
                    records['messages'].append(messages)
                    records['responses'].append(responses)
                    messages = messages_copy
                    responses = responses_copy
                    break
                except Exception as e: 
                    cnt_invalid[q] = cnt_invalid.get(q, 0) + 1
                    if print_except: print(e)
                    messages = messages_copy
                    responses = responses_copy
                    continue
        records['messages'].append(messages)
        records['responses'].append(responses)
        records['choices'].append(choices)
    print('cnt_invalid:', cnt_invalid)
    return records

In [ ]:
switch_model('turbo')
records = run_bigfive(n_instances=30, print_except=False)
save_records('bigfive_turbo', records=records)

In [ ]:
switch_model('gpt4')
records = run_bigfive(n_instances=30, print_except=False)
save_records('bigfive_gpt4', records=records)

## Games

### Dictator Game (1 session, 2 prompts)


In [ ]:
prompts = [
    'Hi, let’s play a game. ',
    'You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). ',
]

unit_test()

In [ ]:
switch_model('gpt-4-0314')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='dictator_gpt4', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_dictator_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/dictator_gpt4_2023_04_05-04_34_28_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

#### Different Models

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=300)
# save_records(game_name='dictator_wo_ex', records=records)
choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_wo_ex = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

switch_model('text-davinci-003')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='dictator_gpt3', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 60.0, 50.0, 70.0, 50.0, 50.0, 70.0, 50.0, 50.0, 75.0, 50.0] + [60, 70, 50, 50, 50, 50, 50, 75, 60, 45, 50, 60, 50, 50]
df_dictator_gpt3 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/dictator_wo_ex_2023_03_31-10_21_50_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

#### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='dictator_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='dictator_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='dictator_gpt4_occupations', records=[records_all, choices_all])

#### Witnessed and Explained

In [ ]:
prompts = [
    'Hi, let’s play a game. ',
    'You are paired with another player. The game host hands you $100. Your role is to decide how to divide this $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]).'
]

unit_test()

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=300)
# save_records(game_name='dictator_witnessed', records=records)
choices = [40.0, 60.0, 60.0, 50.0, 20.0, 20.0, 30.0, 30.0, 50.0, 70.0, 30.0, 30.0, 30.0, 50.0, 30.0] + [30, 20, 40, 40, 40, 70, 40, 40, 30, 70, 60, 40, 50, 40]
df_dictator_wit = choices_to_df(choices, hue=str('%s, witnessed' % MODEL))

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='dictator_gpt4_witnessed', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_dictator_gpt4_wit = choices_to_df(choices, hue=str('%s, witnessed' % MODEL))

In [ ]:
prompts = [
    'Hi, let’s play a game. ',
    'You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). Please explain your choice. ',
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='dictator_gpt4_explained', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_dictator_gpt4_w_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='dictator_explained', records=records)
choices = [40, 60, 50, 70, 50, 50, 30, 50, 70, 50, 40, 30, 80, 70, 40, 30, 30, 75, 30, 50, 30, 50, 50, 30, 40, 40, 50, 30, 50, 20]
df_dictator_w_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

#### Paired

In [ ]:
prompts_paired = lambda x: [
    'Hi, let’s play a game. ',
    'You are paired with a {} player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). '.format(x),
]

# unit_test()

In [ ]:
switch_model('gpt-4-0314')
file_names = {
    'female':   'records/dictator_gpt4_paired_female_2023_04_12-11_10_22_PM.json',
    'male':     'records/dictator_gpt4_paired_male_2023_04_12-11_16_18_PM.json'
}

df_dictator_gpt4_paired = {}
for paired_with in ['female', 'male']:
    # prompts = prompts_paired(paired_with)
    # records = run_one_session(prompts, n_instances=30)
    # save_records(game_name='dictator_gpt4_paired_{}'.format(paired_with), records=records)
    records = json.load(open(file_names[paired_with], 'r'))
    choices = extract_choices(records)
    print(choices)
    df_dictator_gpt4_paired[paired_with] = choices_to_df(choices, hue=str('%s, paired with a {} player'.format(paired_with) % MODEL))

### Ultimatum Game (2 sessions, 2 prompts)

#### Session 1

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
]

unit_test()

In [ ]:
switch_model('gpt-4-0314')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_1_gpt4', records=records)
# choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
# df_ultimatum_1_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/ultimatum_1_gpt4_2023_04_05-04_41_04_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

##### Explained

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]). Please explain your choice. ',
]

unit_test()

In [ ]:
switch_model('gpt-4-0314')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_1_gpt4_explained', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 60.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 40.0, 50.0]
df_ultimatum_1_gpt4_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

##### Different Models

In [ ]:
switch_model('text-davinci-003')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_1_gpt3', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 60.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0] + [50]
df_ultimatum_1_gpt3 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(choices)

In [ ]:
switch_model('gpt-3.5-turbo-0301')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_1', records=records)
# choices = [40, 40, 40, 30, 70, 70, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 30, 30, 35, 50, 40, 70, 40, 60, 60, 70, 40, 50]
# df_ultimatum_1_wo_ex = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/ultimatum_1_wo_ex_2023_03_13-11_54_45_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))
# using extract.ipynb to extract the choices

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_1_explained', records=records)
choices = [30, 30, 40, 40, 50, 60, 40, 30, 40, 40, None, 40, 50, 40, 70, 40, 40, 30, 40, 40, 40, 40, 40, None, 40, 30, 60, 40, 50, 70]
df_ultimatum_1_w_ex = choices_to_df(choices, hue=str('%s, w explanation' % MODEL))

##### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='ultimatum_1_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='ultimatum_1_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='ultimatum_1_gpt4_occupations', records=[records_all, choices_all])

##### Paired

In [ ]:
prompts_paired = lambda x: [
    'Hi, let’s play a game. ',
    'This is a two-player game. You are paired with a {} player. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).'.format(x),
]

prompts_paired('female')
# unit_test()

In [ ]:
switch_model('gpt-4-0314')
file_names = {
    'female':   'records/ultimatum_1_gpt4_paired_female_2023_04_12-11_29_57_PM.json',
    'male':     'records/ultimatum_1_gpt4_paired_male_2023_04_12-11_35_35_PM.json'
}

df_ultimatum_1_gpt4_paired = {}
for paired_with in ['female', 'male']:
    # prompts = prompts_paired(paired_with)
    # records = run_one_session(prompts, n_instances=30)
    # save_records(game_name='ultimatum_1_gpt4_paired_{}'.format(paired_with), records=records)
    records = json.load(open(file_names[paired_with], 'r'))
    choices = extract_choices(records)
    print(choices)
    df_ultimatum_1_gpt4_paired[paired_with] = choices_to_df(choices, hue=str('%s, paired with a {} player'.format(paired_with) % MODEL))

#### Session 2

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
]

unit_test()

In [ ]:
switch_model('gpt-4-0314')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_2_gpt4', records=records)
# choices = [50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 50.0, 25.0, 50.0, 1.0, 1.0, 20.0, 50.0, 50.0, 50.0, 20.0, 50.0, 1.0, 1.0, 1.0, 50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 20.0, 1.0] + [0, 1]
# df_ultimatum_2_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/ultimatum_2_gpt4_2023_04_05-04_49_55_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

In [ ]:
switch_model('gpt4-1106')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_2_gpt4_1106', records=records)

In [ ]:
switch_model('turbo-1106')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_2_turbo_1106', records=records)

##### Explained

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]). Please explain your choice',
]

unit_test()

In [ ]:
switch_model('gpt-4-0314')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_2_gpt4_explained', records=records)
choices = [40.0, 50.0, 40.0, 50.0, 1.0, 20.0, 50.0, 50.0, 1.0, 50.0, 50.0, 20.0, 1.0, 1.0, 20.0, 1.0, 50.0, 1.0, 50.0, 1.0, 1.0, 1.0, 50.0, 20.0, 50.0, 1.0, 50.0, 1.0, 1.0] + [1]
df_ultimatum_2_gpt4_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

##### Different Models

In [ ]:
switch_model('gpt-3.5-turbo-0301')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_2_turbo', records=records)
# choices = [None, 50, 50, 50, 50, 30, None, None, 30, 33.33, 40, None, 50, 40, None, 1, 30, None, 10, 50, 30, 10, 30, None, 30, None, 10, 30, 30, 30]
# df_ultimatum_2_wo_ex = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/ultimatum_2_wo_ex_2023_03_14-12_44_57_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_2_w_ex', records=records)
choices = [50.0, 30.0, 40.0, 40.0, 40.0, 40.0, 50.0, 50.0, 50.0, 30.0, 30.0, 50.0, 40.0, 40.0, 30.0, 1.0, 40.0, 30.0, 50.0] + [1, 30, 50, 30, None, 50, None, 30, 50, None, None]
df_ultimatum_2_w_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

In [ ]:
switch_model('text-davinci-003')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='ultimatum_2_gpt3', records=records)
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 30.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_ultimatum_2_gpt3 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

##### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='ultimatum_2_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='ultimatum_2_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='ultimatum_2_gpt4_occupations', records=[records_all, choices_all])

##### Paired

In [ ]:
prompts_paired = lambda x: [
    'Hi, let’s play a game. ',
    'This is a two-player game. You are paired with a {} player. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).'.format(x),
]

prompts_paired('female')
# unit_test()

In [ ]:
switch_model('gpt-4-0314')
file_names = {
    '':         'records/ultimatum_2_gpt4_paired__2023_06_21-03_10_59_PM.json',
    'female':   'records/ultimatum_2_gpt4_paired_female_2023_04_12-11_52_38_PM.json',
    'male':     'records/ultimatum_2_gpt4_paired_male_2023_04_12-11_58_47_PM.json',
    'chatbot':  'records/ultimatum_2_gpt4_paired_chatbot_2023_06_21-02_44_32_PM.json',
}

df_ultimatum_2_gpt4_paired = {}
for paired_with in ['', 'female', 'male', 'chatbot']:
    # prompts = prompts_paired(paired_with)
    # records = run_one_session(prompts, n_instances=30)
    # save_records(game_name='ultimatum_2_gpt4_paired_{}'.format(paired_with), records=records)
    records = json.load(open(file_names[paired_with], 'r'))
    choices = extract_choices(records)
    print(choices)
    df_ultimatum_2_gpt4_paired[paired_with] = choices_to_df(choices, hue=str('%s, paired with a {} player'.format(paired_with) % MODEL))

#### Session 1 & 2

In [ ]:
prompts = [
    'Hi, let’s play a game.',
    'This is a two-player game. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
    'This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\
     As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]).',
]

unit_test()

In [ ]:
switch_model('gpt4')
records = run_one_session(prompts, n_instances=30)
save_records(game_name='ultimatum_12_gpt4', records=records)

In [ ]:
switch_model('turbo')
records = run_one_session(prompts, n_instances=120)
save_records(game_name='ultimatum_12', records=records)

In [ ]:
# records = json.load(open('records/ultimatum_12_2023_03_29-06_36_56_PM.json', 'r'))
# records = json.load(open('records/ultimatum_21_2023_03_29-05_57_47_PM.json', 'r'))
records = json.load(open('records/ultimatum_12_gpt4_2023_04_07-03_14_30_AM.json', 'r'))
# records = json.load(open('records/ultimatum_21_gpt4_2023_04_07-03_32_21_AM.json', 'r'))

In [ ]:
switch_model('gpt4')
n_instances=30
records = run_one_session(prompts, n_instances=n_instances, orders=[[0,2,1]]*n_instances)
save_records(game_name='ultimatum_21_gpt4', records=records)

In [ ]:
switch_model('turbo')
n_instances=60
records = run_one_session(prompts, n_instances=n_instances, orders=[[0,2,1]]*n_instances)
save_records(game_name='ultimatum_21', records=records)

### Trust Game

#### Session 1

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \
     How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]).'
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_1_gpt4', records=records)
choices = [50.0, 50.0, 40.0, 30.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0, 50.0, 30.0, 30.0, 50.0, 50.0, 50.0, 40.0, 40.0, 50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0] 
df_trust_1_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
records = json.load(open('records/trust_1_gpt4_2023_04_05-05_49_06_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Models

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_1', records=records)
choices = [50.0, 50.0, 30.0, 30.0, 30.0, 60.0, 50.0, 40.0, 20.0, 20.0, 50.0, 40.0, 30.0, 20.0, 30.0, 20.0, 30.0, 60.0, 50.0, 30.0, 50.0, 20.0, 20.0, 30.0, 50.0, 30.0, 30.0, 50.0, 40.0] + [30]
df_trust_1_turbo = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
records = json.load(open('records/trust_1_wo_ex_2023_03_16-11_20_16_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
switch_model('gpt3')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_1_gpt3', records=records)
choices = [30.0, 30.0, 30.0, 30.0, 50.0, 50.0, 100.0, 20.0, 100.0, 50.0, 50.0, 10.0, 20.0, 20.0, 30.0, 30.0, 10.0, 30.0, 50.0, 10.0, 50.0, 100.0, 10.0, 100.0, 20.0, 20.0, 10.0, 20.0, 100.0, 100.0]
df_trust_1_gpt3 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

##### Explained

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \
     How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your choice. '
]

unit_test()

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_1_explained', records=records)
choices = [30.0, 30.0, 50.0, 30.0, 30.0, 70.0, 30.0, 30.0, 30.0, 50.0, 30.0, 30.0, 30.0, 20.0, 30.0, 30.0, 40.0, 30.0, 20.0, 30.0, 30.0, 30.0, 20.0, 30.0, 40.0, 30.0] + [30, None, 30, 100]
df_trust_1_w_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_1_gpt4_explained', records=records)
choices = [50.0, 50.0, 50.0, 40.0, 40.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 60.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 30.0, 50.0, 40.0, 50.0, 50.0] + [50]
df_trust_1_gpt4_w_ex = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

##### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_1_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_1_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_1_gpt4_occupations', records=[records_all, choices_all])

#### Session 2

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). '
]

# unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_2_gpt4', records=records)
choices = [20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.0, 20.0, 15.0]
df_trust_2_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_2_turbo', records=records)
choices = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 25.0, 30.0, 30.0, 20.0, 25.0, 30.0, 20.0, 20.0, 18.0] + [20, 20, 20, 25, 25, 25, 30]
df_trust_2_turbo = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Explained

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.'
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_2_gpt4', records=records)
choices = [20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 15.0, 15.0, 20.0, 20.0]
df_trust_2_gpt4_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Models

In [ ]:
switch_model('gpt3')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_2_gpt3', records=records)
choices = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 25.0, 20.0, 20.0, 20.0, 20.0, 20.0, 25.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
df_trust_2_gpt3_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_2', records=records)
choices = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 30.0, 20.0, 20.0, 30.0, 30.0, 20.0, 20.0] + [None, 20, 10, 20, 20, 20, 20, 20, 25, 20, 20, 20, 20]
df_trust_2_turbo_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

##### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_2_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_2_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_2_gpt4_occupations', records=[records_all, choices_all])

#### Session 3

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $50 with you and the current value became $150, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]).'
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_3_gpt4', records=records)
choices = [100.0, 75.0, 75.0, 75.0, 75.0, 75.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0, 75.0, 75.0, 100.0, 100.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0]
df_trust_3_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
records = json.load(open('records/trust_3_gpt4_2023_04_08-12_03_33_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_3_turbo', records=records)
choices = [150.0, 100.0, 150.0, 150.0, 50.0, 150.0, 100.0, 150.0, 100.0, 100.0, 100.0, 150.0] + [100, 100, 100, 100, 100, 100, 100, 100]
df_trust_3_turbo = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
records = json.load(open('records/trust_3_turbo_2023_04_08-12_05_57_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices if x is not None]))

##### Explained

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $50 with you and the current value became $150, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.'
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_3_gpt4', records=records)
choices = [100.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 100.0, 75.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0]
df_trust_3_gpt4_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Models

In [ ]:
switch_model('gpt3')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_3_gpt3', records=records)
choices = [100.0, 100.0, 120.0, 100.0, 90.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 120.0, 100.0, 100.0, 100.0, 100.0, 100.0, 120.0, 100.0, 100.0, 100.0, 100.0, 90.0, 100.0] + [100, 100, 100]
df_trust_3_gpt3 = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

In [ ]:
switch_model('gpt-3.5-turbo-0301')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_3_w_ex', records=records)
choices = [125.0, 100.0, 100.0, 120.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 100.0, 100.0] + [125, 100, 125, 150, 120, 100, 100, 100, None, 100, None, 100, 120, 100, 100, 100, None, 100]
df_trust_3_turbo_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Roles

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_3_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4-0613')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_3_gpt4_0613_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_3_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo-0613')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_3_turbo_0613_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_3_gpt4_occupations', records=[records_all, choices_all])

#### Session 4

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $100 with you and the current value became $300, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). '
]

# unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_4_gpt4', records=records)
choices = [200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0]
df_trust_4_gpt4 = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_4_turbo', records=records)
choices = [225.0, 225.0, 300.0, 300.0, 220.0, 300.0, 250.0] + [200, 200, 250, 200, 200]
df_trust_4_turbo = choices_to_df(choices, hue=str('%s, w/o explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Explained

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $100 with you and the current value became $300, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.'
]

unit_test()

In [ ]:
switch_model('gpt4')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_4_gpt4', records=records)
choices = [150.0, 200.0, 150.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0]
df_trust_4_gpt4_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Models

In [ ]:
switch_model('gpt3')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_4_gpt3', records=records)
choices = [200.0, 220.0, 200.0, 200.0, 200.0, 210.0, 250.0, 200.0, 240.0, 200.0, 225.0, 200.0, 250.0, 150.0, 200.0, 220.0, 150.0, 200.0, 200.0, 210.0, 240.0, 200.0, 150.0, 200.0, 180.0, 280.0, 200.0, 200.0, 240.0, 200.0]
df_trust_4_gpt3 = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))

In [ ]:
switch_model('turbo')
# records = run_one_session(prompts, n_instances=30)
# save_records(game_name='trust_4_turbo', records=records)
choices = [250.0, 250.0, 200.0, 250.0, 300.0, 300.0, 150.0, 300.0, 300.0] + [None, None, 220, 240, 200, 200, 200, 250, 200, 200, 250, None, None, 200, None, 200, 250, 225, None, 250, 200]
df_trust_4_turbo_explained = choices_to_df(choices, hue=str('%s, w/ explanation' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

##### Different Occupations

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_4_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(prompts=prompts, n_instances=30)
save_records(game_name='trust_4_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(prompts, n_instances=30)
save_records(game_name='trust_4_gpt4_occupations', records=[records_all, choices_all])

#### All Together (with Session 1)

In [ ]:
prompts = [
    'Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \
     How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your choice. ',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $50 with you and the current value became $150, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $100 with you and the current value became $300, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.'
]

unit_test(n_instances=1)

In [ ]:
switch_model('gpt4')
n_instances = 30

orders = [[0, 1, 2]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_gpt4_12', records=records)
records = json.load(open('records/trust_gpt4_12_2023_04_12-01_41_44_AM.json', 'r'))
choices = extract_choices(records)
df_trust_12 = choices_to_df(choices, hue=str('%s, 12' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

orders = [[0, 1, 3]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_gpt4_13', records=records)
records = json.load(open('records/trust_gpt4_13_2023_04_12-02_03_38_AM.json', 'r'))
choices = extract_choices(records)
df_trust_13 = choices_to_df(choices, hue=str('%s, 13' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

orders = [[0, 1, 4]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_gpt4_14', records=records)
records = json.load(open('records/trust_gpt4_14_2023_04_12-02_20_03_AM.json', 'r'))
choices = extract_choices(records)
df_trust_14 = choices_to_df(choices, hue=str('%s, 14' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

orders = [[0, 2, 1]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_21', records=records)
records = json.load(open('records/trust_gpt4_21_2023_04_12-02_35_54_AM.json', 'r'))
choices = extract_choices(records)
df_trust_21 = choices_to_df(choices, hue=str('%s, 21' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

orders = [[0, 3, 1]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_31', records=records)
records = json.load(open('records/trust_gpt4_31_2023_04_12-02_51_38_AM.json', 'r'))
choices = extract_choices(records)
df_trust_31 = choices_to_df(choices, hue=str('%s, 31' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

orders = [[0, 4, 1]] * n_instances
# records = run_one_session(prompts, n_instances=n_instances, orders=orders)
# save_records(game_name='trust_41', records=records)
records = json.load(open('records/trust_gpt4_41_2023_04_12-03_09_46_AM.json', 'r'))
choices = extract_choices(records)
df_trust_41 = choices_to_df(choices, hue=str('%s, 41' % MODEL))
print(', '.join([str(x) for x in choices if x is not None]))

In [ ]:
switch_model('turbo')
n_instances = 30

orders = [[0, 1, 2]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_12', records=records)
choices = extract_choices(records)
df_trust_12 = choices_to_df(choices, hue=str('%s, 12' % MODEL))

orders = [[0, 1, 3]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_13', records=records)
choices = extract_choices(records)
df_trust_13 = choices_to_df(choices, hue=str('%s, 13' % MODEL))

orders = [[0, 1, 4]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_14', records=records)
choices = extract_choices(records)
df_trust_14 = choices_to_df(choices, hue=str('%s, 14' % MODEL))

orders = [[0, 2, 1]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_21', records=records)
choices = extract_choices(records)
df_trust_21 = choices_to_df(choices, hue=str('%s, 21' % MODEL))

orders = [[0, 3, 1]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_31', records=records)
choices = extract_choices(records)
df_trust_31 = choices_to_df(choices, hue=str('%s, 31' % MODEL))

orders = [[0, 4, 1]] * n_instances
records = run_one_session(prompts, n_instances=n_instances, orders=orders)
save_records(game_name='trust_turbo_41', records=records)
choices = extract_choices(records)
df_trust_41 = choices_to_df(choices, hue=str('%s, 41' % MODEL))

### Bomb Risk Game (8 sessions, 5 prompts)

#### Prompts

In [ ]:
prompts_init = [
    'Hi, let\'s play a game.',
    'There are 100 boxes, and one bomb has been randomly placed in 1 of 100 boxes. You can choose to open 0-100 boxes at the same time. If none of the boxes you open has the bomb, you earn points that are equal to the number of boxes you open. If one of the boxes you open has the bomb, you earn zero points.  How many boxes would you open? Please give one concrete number and highlight it with [] (such as [x]).',
]

prompt_congrats = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]). \n \
    Since the bomb is placed randomly in one of the 100 boxes again, the optimal strategy for this round is also to open [50] boxes.' % x

prompt_sorry = 'Sorry! You hit the bomb and earned 0 points in this round. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]).'

prompt_congrats_endgame = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).' % x

prompt_sorry_endgame = 'Sorry! You hit the bomb and earned 0 points in this round. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).'

In [ ]:
extractor = lambda x: '''
Below is a response of a game player, who decide to open a certain number of boxes. How many boxes is this player trying to open? Please output only one number. If there is no answer or the answer is not claer, please output None.

Example:

Input: "Based on this game, I would open [50] boxes to balance the potential reward and the risk involved."

Output: 50

Input: "{}"

Output: 
'''.format(x)

#### Run

In [ ]:
def run_bomb(
    # prompts, 
    n_instances=80, 
    print_except=True,
    only_first=False,
):
    global records
    records = {
        'messages' : [],
        'responses' : [],
        'choices' : [],
        'scenarios': []
    }
    with tqdm(total=n_instances) as pbar:
        while pbar.n < n_instances:
            i = pbar.n
            choices_tmp = []
            scenarios_tmp = []
            try:
                responses = []
                messages = [
                    {"role": "system", "content": SYS_MSG},
                ]

                def extract_bomb():
                    amount = extract_chatgpt(
                        messages[-1]['content'],
                        extractor=extractor,
                        print_except=print_except,
                        type=int,
                    )
                    if amount is None:
                        raise ValueError('Invalid answer: %s' % messages[-1]['content'])
                    choices_tmp.append(amount)
                    return amount

                # round 1
                for prompt in prompts_init:
                    update_messages(messages, responses, prompt)

                amount = extract_bomb()

                if not only_first:

                    # round 2
                    if i % 2 == 1: 
                        scenarios_tmp.append(1)
                        update_messages(messages, responses, prompt_congrats(amount))
                    else: 
                        scenarios_tmp.append(0)
                        update_messages(messages, responses, prompt_sorry)

                    amount = extract_bomb()

                    # round 3
                    if (i // 2) % 2 == 1: 
                        scenarios_tmp.append(1)
                        update_messages(messages, responses, prompt_congrats(amount))
                    else: 
                        scenarios_tmp.append(0)
                        update_messages(messages, responses, prompt_sorry)

                    amount = extract_bomb()

                    # endgame
                    if (i // 4) % 2 == 1: 
                        scenarios_tmp.append(1)
                        update_messages(messages, responses, prompt_congrats_endgame(amount))
                    else: 
                        scenarios_tmp.append(0)
                        update_messages(messages, responses, prompt_sorry_endgame)

                    # amount = extract_bomb()
                
                records['messages'].append(messages)
                records['responses'].append(responses)
                records['choices'].append(choices_tmp)
                records['scenarios'].append(scenarios_tmp)

                pbar.update(1)

            except Exception as e: 
                if print_except: print(e)
                continue
    return records

In [ ]:
unit_test(n_instances=1, run_func=lambda x: run_bomb(n_instances=x, only_first=True))

In [ ]:
switch_model('gpt4')
SYS_MSG = DEFAULT_SYS_MSG
records = run_bomb(n_instances=80, print_except=True)
save_records(game_name='bomb_gpt4', records=records)

In [ ]:
switch_model('turbo')
SYS_MSG = DEFAULT_SYS_MSG
records = run_bomb(n_instances=80, print_except=True)
save_records(game_name='bomb_turbo', records=records)

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(
    run_func=run_bomb,
    extract_func=lambda x: x['choices'],
    n_instances=80,
    only_first=False,
    print_except=True,
)
save_records(game_name='bomb_gpt4_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(
    run_func=run_bomb,
    extract_func=lambda x: x['choices'],
    n_instances=30,
    only_first=False,
    print_except=True,
)
save_records(game_name='bomb_turbo_occupations_described', records=[records_all, choices_all])

In [ ]:
file_names = [
    # 'bomb_gpt4_2023_05_15-12_13_51_AM.json',
    'bomb_turbo_2023_05_14-10_45_50_PM.json'
]

choices = []
scenarios = []
for file_name in file_names:
    with open(os.path.join('records', file_name), 'r') as f:
        records = json.load(f)
        choices += records['choices']
        scenarios += records['scenarios']

assert len(scenarios) == len(choices)
print('loaded %i valid records' % len(scenarios))

In [ ]:
prefix_to_choice = defaultdict(list)
prefix_to_result = defaultdict(list)
prefix_to_pattern = defaultdict(Counter)
# wrong_sum = 0
for scenarios_tmp, choices_tmp in zip(scenarios, choices):
    # try:
    #     result_ref = np.dot(scenarios_tmp, choices_tmp[:-1])
    # except:
    #     continue
    # if not result_ref.item() == choices_tmp[-1]: 
    #         wrong_sum += 1
    #         continue

    result = 0
    for i, scenario in enumerate(scenarios_tmp):
        prefix = tuple(scenarios_tmp[:i])
        choice = choices_tmp[i]
        
        prefix_to_choice[prefix].append(choice)
        prefix_to_pattern[prefix][tuple(choices_tmp[:-1])] += 1

        prefix = tuple(scenarios_tmp[:i+1])
        if scenario == 1:
            result += choice
        prefix_to_result[prefix].append(result)

# print('# of wrong sum:', wrong_sum)
# print('# of correct sum:', len(scenarios) - wrong_sum)

In [ ]:
mean_var_func = lambda x: pd.Series([len(x), np.mean(x), np.std(x), stats.mode(x)])

df_choices = pd.DataFrame(list(prefix_to_choice.items()), columns=['Prefix', 'Choice'])
df_choices[['#samples', 'mean', 'std', 'mode']] = df_choices['Choice'].apply(mean_var_func)
df_choices.sort_values('Prefix')

In [ ]:
for i in range(7):
    print(','.join([str(x) for x in df_choices.iloc[i]['Choice']]))

In [ ]:
switch_model('turbo')
df_bomb_list = []
for i, data in df_choices.iterrows():
    prefix, choices = data['Prefix'], data['Choice']
    df_bomb_list.append(choices_to_df(choices, hue='{}, {}'.format(MODEL, prefix)))
df_bomb_list[0].head()

### Public Goods

#### Basic (1 session, 6 prompts)

In [ ]:
prompt_init = 'Hi, let’s play a game.'

prompt_1 = '''
In this public good game, you and 3 others will choose how much to contribute towards a water cleaning project. Each of you gets $20 per round to contribute between $0 and $20. The project has a 50% return rate. Your payoff relies on both your and others' contributions. Everyone benefits from the group's total contribution. Your payoff in each round equals the amount you didn't contribute (endowment - contribution) plus everyone's benefit (total contributions * 50% return rate). Here are two examples to calculate your payoff.

Example one: You contributed $12; total group contributions were $20

Your Payoff = ($20-$12) + $20*50% = $18

Example two: You contributed $12; total group contributions were $30

Your Payoff = ($20-$12) + $30*50% = $23

We will play a total of 3 rounds, in the first round, how much of the $20 would you like to contribute? Please give a concrete number and highlight it with [] (e.g., [x]).
'''

prompt_2_payoff = lambda x: 'In the first round, the group contribution is ${}, including your contribution. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x)
prompt_2_payoff_explicit = lambda x, y: 'In the first round, the group contribution is ${}, including your contribution. The average payoff for other citizens in the group is ${}. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x,y)

prompt_2 = 'In the second round, you get a new endowment of $20. How much of the $20 would you like to contribute in this round? Please give a concrete number and highlight it with [] (e.g., [x]).'

prompt_3_payoff = lambda x: 'In the second round, the group contribution is ${}, including your contribution. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x)
prompt_3_payoff_explicit = lambda x, y: 'In the second round, the group contribution is ${}, including your contribution. The average payoff for other citizens in the group is ${}. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x,y)

prompt_3 = 'In the third round, you get a new endowment of $20. How much of the $20 would you like to contribute in this round? Please give a concrete number and highlight it with [] (e.g., [x]).'

In [ ]:
def run_PG(
    n_instances=30, 
    print_except=True,
    explicit=False,
    accept_wrong_payoff=True,
    return_rate=.5,
):
    global records
    records = {
        'messages' : [],
        'responses' : [],
        'choices' : [],
        'correct_payoff': []
    }

    with tqdm(total=n_instances) as pbar:
        while pbar.n < n_instances:
            i = pbar.n
            choices_tmp = []
            correct_payoff = True
            try:
                responses = []
                messages = [
                    {"role": "system", "content": SYS_MSG},
                ]

                def extract_contri():
                    amount = extract_amout(
                        messages[-1]['content'], 
                        print_except=print_except, 
                        prefix='$',
                        type=float
                    )
                    if amount is None:
                        raise ValueError('Invalid answer: %s' % messages[-1]['content'])
                    choices_tmp.append(amount)
                    return amount
                
                def extract_payoff():
                    try:
                        amount = extract_amout(
                            messages[-1]['content'], 
                            print_except=print_except, 
                            prefix='$',
                            type=float,
                            brackets='{}'
                        )
                        if amount is None:
                            raise ValueError('Invalid answer: %s' % messages[-1]['content'])
                    except Exception as e:
                        if accept_wrong_payoff:
                            return -1
                        else: raise e
                    return amount

                # round 1
                update_messages(messages, responses, prompt_init)
                update_messages(messages, responses, prompt_1)
                contri = extract_contri()

                # round 2
                # other_contri = 30
                other_contri = 0
                total_contri = other_contri + contri
                benefit = total_contri * return_rate
                other_avg_payoff = 20 - (1.*other_contri/3) + benefit
                if explicit:
                    update_messages(messages, responses, prompt_2_payoff_explicit(total_contri, other_avg_payoff))
                else: update_messages(messages, responses, prompt_2_payoff(total_contri))
                payoff = extract_payoff()
                correct_payoff = correct_payoff and \
                    math.isclose(payoff, (20-contri) + benefit)
                
                update_messages(messages, responses, prompt_2)
                contri = extract_contri()

                # round 3
                # other_contri = 18
                other_contri = 0
                total_contri = other_contri + contri
                benefit = total_contri * return_rate
                other_avg_payoff = 20 - (1.*other_contri/3) + benefit
                benefit = total_contri * return_rate
                if explicit:
                    update_messages(messages, responses, prompt_3_payoff_explicit(total_contri, other_avg_payoff))
                else: update_messages(messages, responses, prompt_3_payoff(total_contri))
                payoff = extract_payoff()
                correct_payoff = correct_payoff and \
                    math.isclose(payoff, (20-contri) + benefit)
                
                update_messages(messages, responses, prompt_3)
                contri = extract_contri()

                if not accept_wrong_payoff and not correct_payoff:
                    raise ValueError('wrong payoff')
                
                records['messages'].append(messages)
                records['responses'].append(responses)
                records['choices'].append(choices_tmp)
                records['correct_payoff'].append(correct_payoff)
                
                pbar.update(1)

            except Exception as e: 
                if print_except: 
                    print(e)
                else: continue
    return records

In [ ]:
switch_model('gpt-4-0314')
SYS_MSG = DEFAULT_SYS_MSG
records = run_PG(
    n_instances=30, 
    explicit=False,
    print_except=True, 
    accept_wrong_payoff=True
)
# unit_test(records)
save_records('PG_basic_gpt4', records=records)
# save_records('PG_basic_gpt4_loss', records=records)

In [ ]:
switch_model('turbo')
SYS_MSG = DEFAULT_SYS_MSG
records = run_PG(
    n_instances=30, 
    explicit=False,
    print_except=True, 
    accept_wrong_payoff=True
)
# save_records('PG_basic_turbo', records=records)
# save_records('PG_basic_turbo_loss', records=records)

#### Different Occupations

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(
    run_func=run_PG,
    extract_func=lambda x: x['choices'],
    n_instances=30,
    print_except=True,
)
# save_records(game_name='PG_gpt4_occupations_described', records=[records_all, choices_all])
save_records(game_name='PG_basic_gpt4_loss_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(
    run_func=run_PG,
    extract_func=lambda x: x['choices'],
    n_instances=30,
    print_except=True,
)
# save_records(game_name='PG_basic_turbo_occupations_described', records=[records_all, choices_all])
save_records(game_name='PG_basic_turbo_loss_occupations_described', records=[records_all, choices_all])

### Prisoner’s Dilemma (1 session, 6 prompts) 

#### Prompts

In [ ]:
prompt_init = 'Hi, let’s play a fun card game.'

prompt_1 = '''
You're paired with another player, each having a $400 'Push' card and a $300 'Pull' card. Your payoff depends on both players' card choices. Here are the scenarios:

* Both play 'Push': Each earns $400
* You play 'Push', the other player plays 'Pull': You earn $0, the other player earns $700
* You play 'Pull', the other player plays 'Push': You earn $700, the other player earns $0
* Both play 'Pull': Each earns $300

In 5 total rounds, which card would you play in the first round? Please give a concrete choice and highlight it with [] (i.e., [Push] or [Pull]). 
'''

prompt_i = lambda *args: 'In the {} round, you played ‘{}’ and the other player played ‘{}’ card. You earned ${} and the other player earned ${}. Which card would you play in the next round? Please give a concrete choice and highlight it with [] (e.g., [Choice]).'.format(*args)

In [ ]:
extractor = lambda x: '''
Below is a response of a game player, who proposed to play a card to another player. What's the card (Push or Pull) this player proposed to play? Please output only the card. If there is no answer or the answer is not claer, please output None.

Example:

Input: "Based on the payoffs, if I play [Push] and my opponent also plays [Push], I'll earn $400. However, if I play [Pull] and my opponent plays [Push], I'll earn $700. The same is true if I play [Push] and my opponent plays [Pull].

Since the payoffs are higher in the case where I play [Pull], I would choose to play [Pull] in the first round."

Output: Pull

Input: "{}"

Output: 
'''.format(x)

#### Run

In [ ]:
def run_PD(
    n_instances=30, 
    print_except=True,
):
    global records
    records = {
        'messages' : [],
        'responses' : [],
        'choices' : [],
    }

    with tqdm(total=n_instances) as pbar:
        while pbar.n < n_instances:
            i = pbar.n
            choices_tmp = []
            try:
                responses = []
                messages = [
                    {"role": "system", "content": SYS_MSG},
                ]

                def extract_card():
                
                    card = extract_chatgpt(
                        messages[-1]['content'],
                        extractor=extractor,
                        print_except=print_except,
                        type=str,
                    )
                    if card not in ['Push', 'Pull']:
                        raise ValueError(
                            'Invalid answer:\n{}'.format(
                                messages[-1]['content']))
                    choices_tmp.append(card)
                    return card
                    
                
                payoff_dict = {
                    ('Push', 'Push'): 400,
                    ('Push', 'Pull'): 0,
                    ('Pull', 'Push'): 700,
                    ('Pull', 'Pull'): 300
                }

                # round 1
                update_messages(messages, responses, prompt_init)
                update_messages(messages, responses, prompt_1)
                card = extract_card()

                def round_i(ith, card, other_card):
                    payoff = payoff_dict[(card, other_card)]
                    other_payoff = payoff_dict[(other_card, card)]
                    update_messages(messages, responses, prompt_i(
                        ith,
                        card,   other_card,
                        payoff, other_payoff
                    ))

                # round 2-5
                round_i('first', card, 'Pull')
                card = extract_card()
                round_i('second', card, 'Pull')
                card = extract_card()
                round_i('third', card, 'Push')
                card = extract_card()
                round_i('fourth', card, 'Push')
                card = extract_card()

                # round_i('first', card, 'Push')
                # card = extract_card()
                
                records['messages'].append(messages)
                records['responses'].append(responses)
                records['choices'].append(choices_tmp)
                
                pbar.update(1)

            except Exception as e: 
                if print_except: print(e)
                continue
    return records

In [ ]:
switch_model('gpt4')
SYS_MSG = DEFAULT_SYS_MSG
# records = run_PD(n_instances=1, print_except=True)
# unit_test(records)
records = run_PD(n_instances=30, print_except=True)
save_records('PD_gpt4_five_rounds_pull', records=records)
# save_records('PD_gpt4_two_rounds_push', records=records)
# save_records('PD_gpt4_two_rounds_pull', records=records)

In [ ]:
switch_model('turbo')
SYS_MSG = DEFAULT_SYS_MSG
records = run_PD(n_instances=30, print_except=True)
save_records('PD_turbo_five_rounds_pull', records=records)
# save_records('PD_turbo_two_rounds_push', records=records)
# save_records('PD_turbo_two_rounds_pull', records=records)

#### Different Occupations

In [ ]:
switch_model('gpt4')
records_all, choices_all = run_occupations(
    run_func=run_PD,
    extract_func=lambda x: x['choices'],
    n_instances=30,
    print_except=True,
)
save_records(game_name='PD_gpt4_five_rounds_pull_occupations_described', records=[records_all, choices_all])
# save_records(game_name='PD_gpt4_two_rounds_push_occupations_described', records=[records_all, choices_all])
# save_records(game_name='PD_gpt4_two_rounds_pull_occupations_described', records=[records_all, choices_all])

In [ ]:
switch_model('turbo')
records_all, choices_all = run_occupations(
    run_func=run_PD,
    extract_func=lambda x: x['choices'],
    n_instances=30,
    print_except=True,
)
save_records(game_name='PD_turbo_five_rounds_occupations_described', records=[records_all, choices_all])
# save_records(game_name='PD_turbo_two_rounds_push_occupations_described', records=[records_all, choices_all])
# save_records(game_name='PD_turbo_two_rounds_pull_occupations_described', records=[records_all, choices_all])